<b>Local Magnitude Estimator</b><br>
This Notebook is designed to demonstrate how to download a set of waveforms from stations near a catalog earthquake and use them to estimate the local magnitude.


In [1]:
import os
from math import log10
from obspy import UTCDateTime, read, read_inventory
from obspy.geodetics import gps2dist_azimuth
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader
from obspy.clients.fdsn import Client
client = Client("IRIS")

The following code will set the variables for the earthquake catalog search.  If a catalog event is not yet available, the values needed for future code are the origin time and the latitude and longitude.<br><br>
In the predefined example, the values are set for a small earthquake in the Eastern Tennessee Seismic Zone.  The USGS page for this event can be found here:<br> <a href=https://earthquake.usgs.gov/earthquakes/eventpage/se60300191>https://earthquake.usgs.gov/earthquakes/eventpage/se60300191</a>

In [2]:
catstartt = UTCDateTime("2020-01-01")
catendt = UTCDateTime("2020-08-15")
lon = -84
lat = 36.43
rad = 1
cat = client.get_events(starttime=catstartt, endtime=catendt, latitude=lat, longitude=lon, maxradius=rad, minmagnitude=3)

Printing the catalog will show the events which have been returned by the search.  Note what the catalog magnitude value is and what type it is.  The origin time is stored in a variable.

In [3]:
print(cat)

1 Event(s) in Catalog:
2020-01-20T19:12:11.260000Z | +36.431,  -84.027 | 3.8 Mwr


Next we set the variables for the waveform search.

In [4]:
origint = UTCDateTime(cat[0].origins[0].time.datetime)
wfstartt = origint - 10
wfendt = origint + 50
stamaxrad = 1
domain = CircularDomain(latitude=lat, longitude=lon, minradius=0, maxradius=stamaxrad)
restrictions = Restrictions(starttime=wfstartt, endtime=wfendt, channel_priorities=["HH[NE12]", "BH[NE12]"], location_priorities=["", "00", "10"])

The following code performs the waveform search.  It looks for stations from many different FDSN clients, so it can take several minutes to complete.  It will produce a lot of information in pink that describes the different steps through the search process.

In [5]:
mdl = MassDownloader()
mdl.download(domain,restrictions, mseed_storage="waveforms", stationxml_storage="stations")

[2021-04-01 19:55:32,953] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for BGR, EMSC, ETH, GEONET, GFZ, ICGC, INGV, IPGP, ISC, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, SCEDC, TEXNET, UIB-NORSAR, USGS, USP, ORFEUS, IRIS.
[2021-04-01 19:55:41,254] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'ISC' as it does not have 'dataselect' and/or 'station' services.
[2021-04-01 19:55:44,015] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'EMSC' as it does not have 'dataselect' and/or 'station' services.
[2021-04-01 19:55:44,033] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'USGS' as it does not have 'dataselect' and/or 'station' services.
[2021-04-01 19:56:11,207] - obspy.clients.fdsn.mass_downloader - WARNING: Failed to initialize client 'ORFEUS'.
[2021-04-01 19:56:11,207] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 19 client(s): BGR, ETH, GEONET, GFZ, ICGC, INGV, IPGP, KNMI, KOERI, LMU, NC

[2021-04-01 19:58:49,059] - obspy.clients.fdsn.mass_downloader - INFO: Client 'TEXNET' - No data available for request.
[2021-04-01 19:58:49,059] - obspy.clients.fdsn.mass_downloader - INFO: Client 'TEXNET' - No data available.
[2021-04-01 19:58:49,059] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2021-04-01 19:58:49,059] - obspy.clients.fdsn.mass_downloader - INFO: Client 'UIB-NORSAR' - Requesting unreliable availability.
[2021-04-01 19:58:49,451] - obspy.clients.fdsn.mass_downloader - INFO: Client 'UIB-NORSAR' - No data available for request.
[2021-04-01 19:58:49,451] - obspy.clients.fdsn.mass_downloader - INFO: Client 'UIB-NORSAR' - No data available.
[2021-04-01 19:58:49,451] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2021-04-01 19:58:49,451] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - Requesting unreliable availability.
[2021-04-01 19:58:50,224] - obspy.clients.fdsn.mass_downloa

{'BGR': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x218996e3b48>,
 'ETH': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x21899840f88>,
 'GEONET': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2189cbafbc8>,
 'GFZ': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2189978d388>,
 'ICGC': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2189ccc3548>,
 'INGV': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2189cca2b08>,
 'IPGP': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2189cd7a248>,
 'KNMI': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2189cd45d08>,
 'KOERI': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2189ccfa7c8>,
 'LMU': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper 

This code loops through each of the downloaded waveforms.  It removes the instrument response using a defined pre-filter and converts to displacement.  The maximum displacement is determined, along with the distance from the event to the station.  The local magnitude is calculated using the approximate emprical formula from Bullen and Bolt (1985).  The average magnitude for each station-channel is reported at the end.

In [6]:
filt = ( 0.5, 1, 40, 50 )#Before removing the response, the seismogram is primarily 
#filtered between 1 and 40 Hz, but allows some energy out to 0.5 and 50 Hz. 
ms = 0.0
n = 0
for file in os.listdir("waveforms"):
    wavfile = "waveforms/" + file
    stafile = "stations/" + file.split(".")[0] + "." + file.split(".")[1] + ".xml"
    print (wavfile, stafile)
    st = read(wavfile)
    tr = st[0]
    inv = read_inventory(stafile)
    tr.remove_response(inventory=inv, pre_filt=filt, output="DISP")
    disp = abs( tr.max() * 1e6 ) # displacement in microns
    stalat = inv.networks[0].stations[0].latitude
    stalon = inv.networks[0].stations[0].longitude
    evelat = cat[0].origins[0].latitude
    evelon = cat[0].origins[0].longitude
    dist = gps2dist_azimuth(stalat, stalon, evelat, evelon )[0] / 1000 # distance in kilometers
    magl = log10(disp) + 2.56 * log10(dist) - 1.67
    print("displacement =",disp,", distance =",dist,", local mag =",magl)
    ms += magl
    n += 1
print ("\n average magnitude =", ms/n)

waveforms/IM.TKL..BHE__20200120T191201Z__20200120T191301Z.mseed stations/IM.TKL.xml
displacement = 13.6072408883 , distance = 88.6924894426726 , local mag = 4.450360396230746
waveforms/IM.TKL..BHN__20200120T191201Z__20200120T191301Z.mseed stations/IM.TKL.xml
displacement = 5.06989054065 , distance = 88.6924894426726 , local mag = 4.021588906062327
waveforms/N4.T50A.00.HH1__20200120T191201Z__20200120T191301Z.mseed stations/N4.T50A.xml
displacement = 5.28455173336 , distance = 97.67877404091344 , local mag = 4.146896666245962
waveforms/N4.T50A.00.HH2__20200120T191201Z__20200120T191301Z.mseed stations/N4.T50A.xml
displacement = 4.72844533109 , distance = 97.67877404091344 , local mag = 4.098606884658201
waveforms/US.TZTN.00.BH1__20200120T191201Z__20200120T191301Z.mseed stations/US.TZTN.xml
displacement = 18.5875474228 , distance = 44.62745650562296 , local mag = 3.822203556164249
waveforms/US.TZTN.00.BH2__20200120T191201Z__20200120T191301Z.mseed stations/US.TZTN.xml
displacement = 13.1303

Correct. For earthquakes of this size, in this part of the country, it is relatively common for the moment magnitude to be slightly smaller than the local magnitude.